# Table of contents 
1. [Pseudo Code](#pseudo)
2. [Import Packages](#import_models)
3. [Models](#model)
    1. [Model-xneed](#model_xneed)
4. [Data](#data)
    1. [Dummy Data Functions](#make_dummy_data)
        1. [Dummy Data Function 1](#make_dummy_data_1)
        2. [Dummy Data Function 2](#make_dummy_data_2)
        3. [Dummy Data Function 3](#make_dummy_data_3)
    2. [Generate Symmetric Uniform Dummy Data](#generate_sym_dummy_data)
    3. [Generate Uniform Dummy Data with more Improvement](#generate_pos_dummy_data)
    4. [Generate Dummy Data with Original Distribution](#generate_orig_dummy_data)
    5. [Import Dummy Data](#import_dummy_data)




## Pseudo Code <a name="pseudo"></a>

For each need $x_{need} \in [-1,1]$, where $-1$ means the need goes to one treatment and 1 means the need goes to other treatment. 0 means no correlation.
 
For each patient, $$x_{patient} = \frac{\sum_{needs} x_{need} \cdot improvement}{3\cdot (\text{number of needs})} \in [-1,1]$$ where $y_{score} \in \{0,1,2,3\}$ is the scoring of that need 
 


Objective:$$max\frac{\sum_{j \in \text{IBHS patients}}x_{patient}\cdot (-1)\cdot (\text{improvement})}{|\text{IBHS patients}|} + \frac{\sum_{j \in \text{FBMHS patients}}x_{patient} \cdot (1) \cdot (\text{improvement})}{|\text{FBMHS patients}|}$$


$x_{need}$ values correspond to correlation with proper treatment.

$-1$: IBHS

$1$: FBMHS


## Import Packages <a name="import_models"></a>

In [1]:
#import the modules
from gurobipy import *
import gurobipy as gp
from gurobipy import GRB
import os
import numpy as np
import pandas as pd
import csv 
import random
import sys
import matplotlib.pyplot as plt

In [2]:
#Some useful settings
#np.set_printoptions(threshold=sys.maxsize)
# %pprint
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_rows', 10)
#pd.set_option('display.max_columns', None)

### Models<a name="model"></a>

#### Model-xneed <a name="model_xneed"></a>

In [3]:
def twosum_improvement_xneed(IBHS, FBMHS, groups_num = 10):  
    
    IBHS_num = IBHS.shape[0]
    FBMHS_num = FBMHS.shape[0]
    numneed = IBHS.shape[1]
    #print('there are ', str(numneed), ' needs.')
    #print('there are ', str(IBHS_num), ' IBHS Patients.')
    #print('there are ', str(FBMHS_num), ' FBMHS Patients.')

    FBMHS = FBMHS.to_numpy()
    selected_rows = np.random.choice(FBMHS_num, size=FBMHS_num, replace=False)
    FBMHS = FBMHS[selected_rows, :]
    
    IBHS = IBHS.to_numpy()
    selected_rows = np.random.choice(IBHS_num, size=IBHS_num, replace=False)
    IBHS = IBHS[selected_rows, :]
    #print(selected_rows)
    ibhs_batch_size = IBHS_num//groups_num
    fbmhs_batch_size = FBMHS_num//groups_num
    
    ibhs_batch = None
    fbmhs_batch = None
     
    res = np.zeros((groups_num, numneed))
    
    for k in range(groups_num):        
        if k < groups_num-1:
            ibhs_batch = IBHS[k*ibhs_batch_size:(k+1)*ibhs_batch_size,]
            fbmhs_batch = FBMHS[k*fbmhs_batch_size:(k+1)*fbmhs_batch_size,]
        else:
            ibhs_batch = IBHS[k*ibhs_batch_size:,]
            fbmhs_batch = FBMHS[k*fbmhs_batch_size:,]

        #Gurobi model                
        m = gp.Model()
        m.Params.LogToConsole = 0 #Gurobi would not print

        #Create decision variables
        x_need = m.addVars(numneed, lb = -1, ub = 1, vtype = GRB.CONTINUOUS, name = 'x_need')

        #Find x weighted score
        x_patient_IBHS = [0 for i in range(ibhs_batch.shape[0])]
        for i in range(ibhs_batch.shape[0]):
            x_patient_IBHS[i] = sum(x_need[j]*ibhs_batch[i][j] for j in range(numneed))/(3*numneed)

        x_patient_FBMHS = [0 for i in range(fbmhs_batch.shape[0])]
        for i in range(fbmhs_batch.shape[0]):
            x_patient_FBMHS[i] = sum(x_need[j]*fbmhs_batch[i][j] for j in range(numneed))/(3*numneed)


        #Using given data to find total improvement for each patient
        x_improvement_IBHS = [0 for i in range(ibhs_batch.shape[0])]
        x_improvement_FBMHS = [0 for i in range(fbmhs_batch.shape[0])]

        for i in range(ibhs_batch.shape[0]):
            x_improvement_IBHS[i] = sum(ibhs_batch[i][j] for j in range(numneed))

        for i in range(fbmhs_batch.shape[0]):
            x_improvement_FBMHS[i] = sum(fbmhs_batch[i][j] for j in range(numneed))

        ibhs_sum = gp.quicksum(x_patient_IBHS[i]*(-1)*x_improvement_IBHS[i] for i in range(ibhs_batch.shape[0]))
        fbmh_sum = gp.quicksum(x_patient_FBMHS[i]*x_improvement_FBMHS[i] for i in range(fbmhs_batch.shape[0]))
        m.setObjective(ibhs_sum/(ibhs_batch.shape[0]) + fbmh_sum/fbmhs_batch.shape[0],
                       GRB.MAXIMIZE)

        m.optimize()
        
        res[k,:] = [x_need[i].X for i in range(numneed)]
        
    return np.sum(res, axis = 0)/groups_num

### Dummy Data Functions <a name="make_dummy_data"></a>

#### Dummy Data Function 1 <a name="make_dummy_data_1"></a>

In [4]:
def make_dummy_data_1(fields, num_needs, high_needs,num_patient, if_succ, file_name):
    rows = []
    for i in range(num_patient):
        rownum = i*2
        #ma number
        row0 = [str(i)]
        row1 = [str(i)]
        #program
        # program = random.choice(['IBHS','FBMHS'])
        if i < num_patient//2:
            program = 'IBHS'
        else:
            program = 'FBMHS'
        row0.append(program)
        row1.append(program)
        
        #if_success = random.choice(['Success','Not Success'])
        if if_succ:
            if_success = 'Success'
        else: 
            if num <= .2:
                if_success = 'Not Success'
            else:
                if_success = 'Success'
        row0.append('A-INITIAL')
        row0.append(if_success)
        row1.append('DISCHARGE')
        row1.append(if_success)
        #needs
        for ni in range(num_needs):
            if (ni) in high_needs[program]:
                need = random.choice([1,2,3])
                row0.append(need)
                if if_success == 'Success':
                    improvement = random.choice([1,2,3])
                    row1.append(max(need-improvement,0))
                else:
                    improvement = random.choice([0,1,2,3])
                    row1.append(min(3,need+improvement))
            elif ni in high_needs['neutral']:
                need = random.choice([1,2,3])
                row0.append(need)
                if if_success == 'Success':
                    improvement = random.choice([0,1,2,3])
                    #improvement = 1
                    row1.append(max(need-improvement,0))
                else:
                    improvement = random.choice([0,1,2,3])
                    row1.append(min(3,need+improvement))
            else:
                need = random.choice([0,1,2,3])
                #need = 0
                row0.append(need)
                if if_success == 'Success':
                    #improvement = random.choice([0])
                    improvement = 0
                    row1.append(max(need-improvement,0))
                else:
                    improvement = random.choice([0,1,2,3])
                    row1.append(min(3,need+improvement))
        rows.append(row0)
        rows.append(row1)

    # name of csv file
    filename = file_name

    # writing to csv file 
    with open(filename, 'w') as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 

        # writing the fields 
        csvwriter.writerow(fields) 

        # writing the data rows 
        csvwriter.writerows(rows)


#### Dummy Data Function 2 <a name="make_dummy_data_2"></a>

For dummy data_2:

- need 0-29: 70% of ibhs_patients improve(1,2,3), 30% worse/stay the same; 30% fbmhs_patient improve(1,2,3), 70% worse/stay the same

- need 30-59: 70% of fbmhs_patient improve(1,2,3), 30% worse/stay the same; 30% ibhs_patients improve(1,2,3), 70% worse/stay the same

- need 60-99: completely random

In [5]:
def make_dummy_data_2(fields, num_needs, num_patient, improv_ratio = 0.7, 
                      file_name = "dummy_data_test.csv"):
    rows = []
    for i in range(num_patient):
        rownum = i*2
        #ma number
        row0 = [str(i)]
        row1 = [str(i)]
        #program
        if i < num_patient//2:
            program = 'IBHS'
        else:
            program = 'FBMHS'
        
        row0.append(program)
        row1.append(program)
        #reason for terminate
        
        row0.append('A-INITIAL')        
        row1.append('DISCHARGE')

        #needs
        
        for ni in range(num_needs):
            if ni <= 29:
                if program == 'IBHS':
                    num = random.random()
                    if num <= improv_ratio:
                        need = random.choice([1,2,3])
                        row0.append(need)

                        improvement = random.choice([1,2,3])
                        row1.append(max(0,need-improvement))
                    else:
                        need = random.choice([0,1,2,3])
                        row0.append(need)

                        improvement = random.choice([0,1,2,3])
                        row1.append(min(3,need+improvement))
                else:
                    num = random.random()
                    if num < 1- improv_ratio:
                        need = random.choice([1,2,3])
                        row0.append(need)

                        improvement = random.choice([1,2,3])
                        row1.append(max(0,need-improvement))
                    else:
                        need = random.choice([0,1,2,3])
                        row0.append(need)

                        improvement = random.choice([0,1,2,3])
                        row1.append(min(3,need+improvement))
            elif ni >= 30 and ni <= 59:
                if program == 'FBMHS':
                    num = random.random()
                    if num <= improv_ratio:
                        need = random.choice([1,2,3])
                        row0.append(need)

                        improvement = random.choice([1,2,3])
                        row1.append(max(0,need-improvement))
                    else:
                        need = random.choice([0,1,2,3])
                        row0.append(need)

                        improvement = random.choice([0,1,2,3])
                        row1.append(min(3,need+improvement))
                else:
                    #IBHS
                    num = random.random()
                    if num < 1-improv_ratio:
                        need = random.choice([1,2,3])
                        row0.append(need)

                        improvement = random.choice([1,2,3])
                        row1.append(max(0,need-improvement))
                    else:
                        need = random.choice([0,1,2,3])
                        row0.append(need)

                        improvement = random.choice([0,1,2,3])
                        row1.append(min(3,need+improvement))
            else:
                num = random.random()
                if num <= .5:
                    need = random.choice([1,2,3])
                    row0.append(need)

                    improvement = random.choice([1,2,3])
                    row1.append(max(0,need-improvement))
                else:
                    need = random.choice([0,1,2,3])
                    row0.append(need)

                    improvement = random.choice([0,1,2,3])
                    row1.append(min(3,need+improvement))
        rows.append(row0)
        rows.append(row1)
        
    
    # writing to csv file 
    with open(file_name, 'w') as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 

        # writing the fields 
        csvwriter.writerow(fields) 

        # writing the data rows 
        csvwriter.writerows(rows)
                
                    

#### Dummy Data Function 3 <a name="make_dummy_data_3"></a>

In [6]:
def find_distribution(df):
    dis = " "
    for i in range(df.shape[1]):
        if i == 0:
            dis =  df[str(0)].value_counts()
        else:
            cur =  df[str(i)].value_counts()
            for j in range(7):
                k = j - 3
                if k not in cur:
                    cur[k] = 0
            dis +=  cur
    for j in range(7):
        k = j - 3
        if k not in dis:
            # print(k)
            dis[k] = 0
    return dis

In [7]:
ibhs_df = pd.read_csv('ibhs_all_improv_table.csv')
fbmh_df = pd.read_csv('fbmhs_all_improv_table.csv')

ibhs_dis = find_distribution(ibhs_df)
fbmh_dis = find_distribution(fbmh_df)

In [8]:
ibhs_dis

 0    270431.0
-1     25553.0
 1     39052.0
-2      6694.0
 2     14419.0
-3      1118.0
 3         0.0
Name: 0, dtype: float64

In [9]:
'''
 1. generator that outputs improvement and disimprovement with values based on distribution(uniform, original) 
 2. whether the output disimproves and improves is predetermined by if_improv
'''
def random_generator(dis, lst, program = 'IBHS', if_improv = True):
    if dis == 'uniform':
        return random.choice(lst)
    if if_improv and dis == 'original':
        if program == 'IBHS':
            num = random.random()
            total = ibhs_dis[0] + ibhs_dis[1] + ibhs_dis[2] + ibhs_dis[3]
            if num < ibhs_dis[0]/total:
                return 0
            elif num < ibhs_dis[1]/total:
                return 1
            elif num < ibhs_dis[2]/total:
                return 2
            else:
                return 3
        elif program == 'FMBHS':
            num = random.random()
            total = fbmh_dis[0] + fbmh_dis[1] + fbmh_dis[2] + fbmh_dis[3]
            if num < fbmh_dis[0]/total:
                return 0
            elif num < fbmh_dis[1]/total:
                return 1
            elif num < fbmh_dis[2]/total:
                return 2
            else:
                return 3
    elif not if_improv and dis == 'original':
        if program == 'IBHS':
            num = random.random()
            total = ibhs_dis[0] + ibhs_dis[-1] + ibhs_dis[-2] + ibhs_dis[-3]
            if num < ibhs_dis[0]/total:
                return 0
            elif num < ibhs_dis[-1]/total:
                return -1
            elif num < ibhs_dis[-2]/total:
                return -2
            else:
                return -3
        elif program == 'FMBHS':
            num = random.random()
            total = fbmh_dis[0] + fbmh_dis[-1] + fbmh_dis[-2] + fbmh_dis[-3]
            if num < fbmh_dis[0]/total:
                return 0
            elif num < fbmh_dis[-1]/total:
                return -1
            elif num < fbmh_dis[-2]/total:
                return -2
            else:
                return -3
        

In [10]:
'''
1. each item only improves if improve_only = True, and it improves based on the dis (uniform/original)
2. file_name is the output file name
3. need_groups is a list of the cutting points with length 3. 
   For need_groups = [a,b,c], 
   [0:a] are the items that must improve with IBHS(disimproves with FBMHS), 
   [a:b] are the items that must improve with FBMHS(disimproves with IBHS), 
   [b:c] are the items improve randomly
'''
def make_dummy_data_2(fields, num_patient, ratio = 0.7, 
                      improve_only = True, need_groups = [5,10,90], 
                      file_name = 'dummy_data_test', dis = 'uniform',
                     improv = [0,1,2,3], disImprov = [0,-1,-2,-3]):
    rows = []
    for i in range(num_patient):

        #ma number
        row = [str(i)]
        #program
        if i < num_patient//2:
            program = 'IBHS'
        else:
            program = 'FBMHS'
        
        row.append(program)

        #needs
        
        for ni in range(need_groups[-1]):
            if ni < need_groups[0]:
                if program == 'IBHS':
                    if improve_only:
                        improvement = random_generator(dis, improv)
                        row.append(improvement)
                    else:
                        num = random.random()
                        if num <= ratio:
                            improvement = random_generator(dis, improv)
                            row.append(improvement)
                        else:
                            improvement = random_generator(dis, disImprov, if_improv = False)
                            row.append(improvement)
                else: #FBMHS
                    if improve_only:
                        improvement = 0
                        row.append(improvement)
                    else:
                        num = random.random()
                        if num < 1- ratio:
                            improvement = random_generator(dis, improv)
                            row.append(improvement)
                        else:
                            improvement = random_generator(dis, disImprov, if_improv = False)
                            row.append(improvement)
            elif ni >= need_groups[0] and ni < need_groups[1]:
                if program == 'FBMHS':
                    if improve_only:
                        improvement = random_generator(dis, improv)
                        row.append(improvement)
                    else:
                        if improve_only:
                            improvement = random_generator(dis, improv)
                            row.append(improvement)
                        else:
                            num = random.random()
                            if num <= ratio:
                                improvement = random_generator(dis, improv)
                                row.append(improvement)
                            else:
                                improvement = random_generator(dis, disImprov, if_improv = False)
                                row.append(improvement)
                else: #IBHS
                    if improve_only:
                        improvement = 0
                        row.append(improvement)
                    
                    else:
                        if improve_only:
                            improvement = random_generator(dis, improv)
                            row.append(improvement)
                        else:
                            num = random.random()
                            if num < 1-ratio:
                                improvement = random_generator(dis, improv)
                                row.append(improvement)
                            else:
                                improvement = random_generator(dis, disImprov, if_improv = False)
                                row.append(improvement)
            else:
                if improve_only:
                    improvement = random_generator(dis, improv)
                    row.append(improvement)
                else:
                    improvement = random_generator(dis, list(set(disImprov + improv)), if_improv = random.choice([False, True]))
                    row.append(improvement)
        rows.append(row)
        
    filename = file_name
    # writing to csv file 
    with open(filename, 'w') as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 

        # writing the fields 
        csvwriter.writerow(fields) 

        # writing the data rows 
        csvwriter.writerows(rows)
                
                    

### Generating Symmetric Dummy Data <a name="generate_sym_dummy_data"></a>


In [12]:
num_patient = 30
need_splits = [5,10,90]
fields_need = [str(i) for i in range(need_splits[-1])]
fields = ['client_id', 'Program'] + fields_need
name = 'dummy_group_1.csv'
make_dummy_data_2(fields, num_patient, ratio = 1, 
                  improve_only = True, need_groups = need_splits, 
                  file_name = name)


### Generate Uniform Dummy Data with more Improvement  <a name="generate_pos_dummy_data"></a>

In [13]:
num_patient = 30
needs_split = [5,10,90]
fields_need = [str(i) for i in range(needs_split[-1])]
fields = ['client_id', 'Program'] + fields_need
name = 'dummy_group_1_more_improv.csv'
make_dummy_data_2(fields, num_patient, ratio = 0.7, 
                      improve_only = True, need_groups = needs_split, 
                      file_name = name, dis = 'uniform',
                      improv = [0,1,2,3], disImprov = [0,-1] )


### Generate Dummy Data with Original Distribution <a name="generate_orig_dummy_data"></a>

In [14]:
num_patient = 30
needs_split = [5,10,90]
fields_need = [str(i) for i in range(needs_split[-1])]
fields = ['client_id', 'Program'] + fields_need
name = 'dummy_group_1_origin.csv'
make_dummy_data_2(fields, num_patient, ratio = 0.7, 
                      improve_only = True, need_groups = needs_split, 
                      file_name = name, dis = 'original')


### Import Dummy Data <a name="import_dummy_data"></a>

In [15]:
def import_data(name):
    test_dummy = pd.read_csv(name)
    test_dummy_ibhs = test_dummy[test_dummy['Program'] == 'IBHS']
    test_dummy_fbmh = test_dummy[test_dummy['Program'] == 'FBMHS']
    droplist = ['client_id','Program']
    test_dummy_fbmh = test_dummy_fbmh.reset_index(drop=True)

    test_dummy_fbmh.drop(droplist, axis = 1, inplace = True)
    test_dummy_ibhs.drop(droplist, axis = 1, inplace = True)
    return (test_dummy_ibhs, test_dummy_fbmh)